<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/rnn_many_to_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
%matplotlib inline

print(tf.__version__)
tf.enable_eager_execution()

1.12.0


In [7]:
# 데이터 준비. 실제 쓰이는 단어와 그 단어에 좋다 나쁘다  태깅을 한 것
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]

# 토큰 딕쇼너리를 만든다
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
#글자 단위로 글자 세트 생성
idx2char = {idx : char for idx, char in enumerate(char_set)}
# enumerate는 인덱스를 붙인 시퀀스로 생성해 줌 for문과 같이 쓰여서 [인덱스, 원래 원소] 이걸 순차적으로 반복해 줌
# :를 써도 되는 진 몰랐는데 딕셔너리 생성
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(char_set)
print(idx2char)
print(char2idx)


['<pad>', ' ', 'a', 'b', 'd', 'e', 'g', 'o', 'r', 's', 'w']
{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [12]:
# 단어들을 만들어진 캐릭터 벡터 인덱스로 치환함
x_data = list(map(lambda word : [char2idx.get(char) for char in word], words))
# map 함수는 앞에는 함수, 뒤에는 인자를 받는다. 
# lambda 함수는 인스턴트 함수를 만들어 준다. 여기서 이용되는 것은 word를 받아서 word에 있는 걸 char로 받아서 전개
# 적용되는 함수는 char2idx.get(char)인데, ()안에 인자로 key 값을 넣으면 value를 반환해주는 함수이다.
# lambda word는 words의 스펠링 하나하나를 char로 받아 char2idx를 통해 key->value로 치환하고 그걸 리스트로 만든다
x_data_len = list(map(lambda word : len(word), x_data))
# x_data를 인자(word)로 받아서 각 단어들을 len에다 집어넣은 값 출력

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [13]:
# 
max_sequence = 10
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence,
                      padding = 'post', truncating = 'post')
# pad_sequences는 함수로 keras의 전처리를 위한 함수, 기초 인자는 시퀀스 데이터, 최대 시퀀스 길이, 등등..?
# padding은 하라는 거겠지만 post나 truncate post는 무슨 뜻인지 모르겠당
# padding: 패딩을 앞에 할 것인가(pre), 뒤로 할 것인가(post). 여기서는 뒤로 하기 위해 post
# truncating: 넘치는 글자를 어쩔것인가? 앞을 자를 거면 'pre', 뒤를 자를 거면 'post'
# 패딩 달아주는거. 기본 시퀀스 투입하고 최대 길이 정해주고, 패딩 어디 쌓을 건지 정하고(양옆은 없네), 넘치면 어떻게 자를건지 정해줌)

print(x_data)
print(x_data_len)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[4, 3, 5, 7]
[1, 0, 0, 1]


In [0]:

input_dim = len(char2idx)
#사전의 크기
output_dim = len(char2idx)
# 역시 사전의 크기
one_hot = np.eye(len(char2idx))
#  len(char2idx) * len(char2idx) 크기의 항등행렬을 만든다,

hidden_size = 10
num_classes = 2

model = Sequential()
# Sequential 객체 선언
model.add(layers.Embedding(input_dim = input_dim, output_dim = output_dim,
                           trainable = False, mask_zero = True, input_length = max_sequence,
                           embeddings_initializer = keras.initializers.Constant(one_hot)))
# keras 특유의 레이어 쌓기. 근데 인풋과 아웃풋 차원이 이미 정해져있네..
# rnn은 원래 한 시퀀스마다 결과를 뱉으니까 가능할 듯
# 
model.add(layers.SimpleRNN(units = hidden_size))
model.add(layers.Dense(units = num_classes))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 11)            121       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 10)                220       
_________________________________________________________________
dense (Dense)                (None, 2)                 22        
Total params: 363
Trainable params: 242
Non-trainable params: 121
_________________________________________________________________
